In [5]:
#SVN is used to classify for sparse datasets with binary classification
#downloading stuff
#install stuff
!pip install opencv-python
!pip install torch torchvision pycocotools matplotlib scikit-learn pandas

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [ ]:
#References
#1. https://www.geeksforgeeks.org/machine-learning/classifying-data-using-support-vector-machinessvms-in-python/

In [6]:
#importing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [10]:
#troubleshooting
import os
print(os.getcwd())
print(os.listdir('.'))  # lists files/folders in current directory
print(os.listdir('100_annotated_pins_for_model'))  # lists files in your folder


/home/jovyan
['.config', '.local', 'Image Augmentation', '.jupyter', 'model_experimentation', 'shared', 'Github Repo', 'Untitled.ipynb', 'Annotated_ImageSets', '.ipynb_checkpoints', '.keras', '.ipython', 'SVN Binary Classificer', 'UNET Segmentatoin', 'Untitled Folder', '.cache']


FileNotFoundError: [Errno 2] No such file or directory: '100_annotated_pins_for_model'

In [16]:
import os

# Look for CSV files under the base directory
for root, dirs, files in os.walk('/home/jovyan'):
    for file in files:
        if file.endswith('.csv'):
            print(os.path.join(root, file))


/home/jovyan/SVN Binary Classificer/100_annotated_pins_for_modeldata/train_data.csv
/home/jovyan/SVN Binary Classificer/100_annotated_pins_for_modeldata/validtn_data.csv
/home/jovyan/UNET Segmentatoin/100_annotated_pins_for_model/data/validtn_data.csv
/home/jovyan/UNET Segmentatoin/100_annotated_pins_for_model/data/train_data.csv
/home/jovyan/UNET Segmentatoin/data/validtn_data.csv
/home/jovyan/UNET Segmentatoin/data/train_data.csv
/home/jovyan/UNET Segmentatoin/100_annotated_pins_for_modeldata/train_data.csv
/home/jovyan/UNET Segmentatoin/100_annotated_pins_for_modeldata/validtn_data.csv


In [17]:


# Correct paths
train_csv_path = '/home/jovyan/SVN Binary Classificer/100_annotated_pins_for_modeldata/train_data.csv'
valid_csv_path = '/home/jovyan/SVN Binary Classificer/100_annotated_pins_for_modeldata/validtn_data.csv'

# Load CSVs
train_df = pd.read_csv(train_csv_path)
valid_df = pd.read_csv(valid_csv_path)

# Check the first few rows and columns
print("Train CSV columns:", train_df.columns)
print(train_df.head())
print("Valid CSV columns:", valid_df.columns)
print(valid_df.head())

# Prepare image file paths
X_train = train_df['filepath'].values
X_valid = valid_df['filepath'].values

# If you have masks in CSV, uncomment and adjust these lines:
# y_train = train_df['rle_1'].values  # adjust column names if needed
# y_valid = valid_df['rle_1'].values

print("Number of training samples:", len(X_train))
print("Number of validation samples:", len(X_valid))


Train CSV columns: Index(['filepath'], dtype='object')
                                            filepath
0  100_annotated_pins_for_model/IMG_7046_flat_02_...
1  100_annotated_pins_for_model/IMG_7392_flat_00_...
2  100_annotated_pins_for_model/IMG_7311_flat_00_...
3  100_annotated_pins_for_model/IMG_7043_flat_00_...
4  100_annotated_pins_for_model/IMG_4875_flat_00_...
Valid CSV columns: Index(['filepath'], dtype='object')
                                            filepath
0  100_annotated_pins_for_model/IMG_2937_flat_01_...
1  100_annotated_pins_for_model/IMG_2934_flat_01_...
2  100_annotated_pins_for_model/IMG_7910_flat_00_...
3  100_annotated_pins_for_model/IMG_5262_flat_00_...
4  100_annotated_pins_for_model/IMG_4874_flat_01_...
Number of training samples: 175
Number of validation samples: 31


In [23]:
import os
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

# Base folder where images actually exist
base_dir = '/home/jovyan/SVN Binary Classificer/100_annotated_pins_for_model2'

# Fix CSV paths
def fix_path(csv_path):
    # Take only the filename from the CSV path
    filename = os.path.basename(csv_path)
    return os.path.join(base_dir, filename)

X_train_paths = train_df['filepath'].apply(fix_path)
X_valid_paths = valid_df['filepath'].apply(fix_path)

# Parameters
img_height, img_width = 128, 128

# Load and preprocess training images
num_train = len(X_train_paths)
X_train_array = np.zeros((num_train, img_height, img_width, 3), dtype=np.float32)

for i, path in enumerate(X_train_paths):
    if os.path.exists(path):
        img = load_img(path, target_size=(img_height, img_width))
        X_train_array[i] = img_to_array(img) / 255.0
    else:
        print(f"File not found: {path}")

# Load and preprocess validation images
num_valid = len(X_valid_paths)
X_valid_array = np.zeros((num_valid, img_height, img_width, 3), dtype=np.float32)

for i, path in enumerate(X_valid_paths):
    if os.path.exists(path):
        img = load_img(path, target_size=(img_height, img_width))
        X_valid_array[i] = img_to_array(img) / 255.0
    else:
        print(f"File not found: {path}")

print("Training shape:", X_train_array.shape)
print("Validation shape:", X_valid_array.shape)


Training shape: (175, 128, 128, 3)
Validation shape: (31, 128, 128, 3)


In [25]:
# Assuming your CSV has a column named 'label' with numeric values
y_train = train_df['label'].values
y_valid = valid_df['label'].values

# Flatten images
X_train_flat = X_train_array.reshape(len(X_train_array), -1)
X_valid_flat = X_valid_array.reshape(len(X_valid_array), -1)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flat)
X_valid_scaled = scaler.transform(X_valid_flat)



NameError: name 'y_train' is not defined

In [ ]:
# Train SVM
svm_classifier = SVC(kernel='linear', C=1.0, random_state=42)
svm_classifier.fit(X_train_scaled, y_train)

In [ ]:
#See Metrics (add confusion matrix and so many more)
# Evaluate
accuracy = svm_classifier.score(X_valid_scaled, y_valid)
print("Validation Accuracy:", accuracy)